### Import data

In [10]:
using Pkg
Pkg.instantiate()
using DrugResponseModel

# import data from the path
conc_l, pop, g2, g1, g2_0, g1_0 = setup_data("lapatinib");# in which:
# pop: population data
# g1, g2: g1 and g2 data
# initial: initial number of cells in g1 and in g2 at time 0

### Estimate 

In [2]:
# ODE optimization and estimation of the parameters
params_ode = zeros(4, 8)
for i in 1:8
    fit, params_ode[:, i] = ODEoptimizer([1.0, 1.0, 0.0, 0.0], i, g1, g2, g1_0, g2_0)
    print(fit, "\n")
end

271.6207365366957
382.0181232554082
611.0577455770123
1477.4400586352356
2854.240458062539
2316.4312300068477
652.3421356247128
665.8307514768001


### Turing

In [11]:
i=5
chain = turing(params_ode, g1_0, g2_0, i)

Array{Float64,1}[[13.286044850033335, 6.713955149966666], [6.005905015554943, 3.3113433473652103], [2.805698135494574, 1.5497893756846037], [1.3116441185353713, 0.724544522588782], [0.613193950869968, 0.33872503568764245], [0.28666836955954467, 0.15835406629641008], [0.13401760044251915, 0.07403060207466183], [0.06265325835710338, 0.03460932312506627], [0.02929045026497974, 0.016179887276120547], [0.01369330918664748, 0.0075641103865828905], [0.006401622597018234, 0.003536221910793274], [0.002992741553878479, 0.0016531743469375985], [0.0013991236350342968, 0.0007728683750306525], [0.0006541060806198383, 0.0003613246827996921], [0.00030579053125456054, 0.00016891704569389594], [0.00014295794307828962, 7.89691992889869e-5], [6.684300106116468e-5, 3.692371447465967e-5], [3.1259596324105926e-5, 1.7267632963357992e-5], [1.4602984356483792e-5, 8.066610117161498e-6], [6.8224078310799195e-6, 3.76872162962765e-6], [3.2054650539528746e-6, 1.7706090314408557e-6], [1.4687578067089577e-6, 8.3335539

BoundsError: BoundsError: attempt to access 2×192 Array{Float64,2} at index [Base.Slice(Base.OneTo(2)), 193]

In [4]:
# plot(sol)
# scatter!(tp,newsol[1,:])
# scatter!(tp,newsol[2,:])
# using StatsPlots
# plot(chain)

In [5]:
# plotting the fitted curves
# ODEplot_all(params_ode, g1, g2, g1_0, g2_0, pop)